# Beeinträchtigungen im Studium

## Imports

In [50]:
import pandas as pd
import plotly.io as pio
import plotly.express as px

## Template

In [51]:
year = '2024'
infoviz_template = dict(
    layout=dict(
        template="plotly_white",
        title=dict(
            font=dict(size=20, family="Arial", weight="bold", color="black"),
            xanchor="left",  
            xref="paper",
            x=0,
            subtitle=dict(
                text=f"absolute Verteilung der Beeinsträchtigungen",
                font=dict(color="gray", size=13),
            ),
        ),
        xaxis=dict(
            showgrid=False,
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelstandoff=10,
            ticklabelposition="outside bottom"
        ),
        yaxis=dict(
            showgrid=True, gridcolor="lightgrey",
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelstandoff=10,
            ticklabelposition="outside left"
        ),
    )
)
pio.templates["infoviz"] = infoviz_template


In [52]:
df = pd.read_excel("data.xlsx", sheet_name=year)

# Filter relevant rows
df = df[
    (df["Variable"] == "Studienbeeinträchtigungen")
    | (df["Category"] == "Anzahl Teilnahmen an Studierendenbefragung")
    | (df["Category"] == "Anteil Teilnehmer (%)")
]

# Drop columns with '*' in name
df = df[df.columns[~df.columns.str.contains(r"\*")]]

# Remove '*' from remaining column names (just in case)
df.columns = df.columns.str.replace("*", "", regex=False)

# Fix typo in column name
df = df.rename(columns={"Master Getränketechnologi": "Master Getränketechnologie"})

# Build a dictionary mapping cleaned column names to original columns
clean_columns = {}
for col in df.columns:
    clean_name = col.strip()  # remove whitespace
    if clean_name in clean_columns:
        clean_columns[clean_name].append(col)
    else:
        clean_columns[clean_name] = [col]

In [53]:
# Sum numerical values row-wise
df["sum"] = df.sum(axis=1, numeric_only=True)

# Keep only relevant rows and columns
df = df.loc[
    ~df["Category"].isin([
        "Anzahl Teilnahmen an Studierendenbefragung",
        "Anteil Teilnehmer (%)"
    ]),
    ["Category", "sum"]
]

# Clean category names by removing ' (%)'
df["Category"] = df["Category"].str.replace(" (%)", "", regex=False)

In [54]:
# Rename long or inconsistent category labels
umbenennungen = {
    "zeitliche Überschneidungen": "Zeitmanagement-Probleme",
    "Stundenplanprobleme wegen weiter Wege": "Wegeprobleme",
    "Organisation des Prüfungsablaufes": "Prüfungsorganisation",
    "Engpässe in der Literaturversorgung": "Literaturmangel",
    "kein Labor- / Praktikumsplatz bekommen": "Fehlender Praktikum-/Laborsplatz",
    "Anforderungen sind zu hoch": "Überforderung",
    "Leistungsnachweise / Prüfungen nicht geschafft": "Prüfungsprobleme",
    "Job / Erwerbstätigkeit": "Erwerbstätigkeit",
    "familiäre Verpflichtungen": "Familienpflichten",
    "Krankheit / Behinderung": "Gesundheitsprobleme",
    "große Entfernung zwischen Wohnort und JLU": "Entfernung zum Studienort",
    "Doppelstudium": "Doppelte Studienbelastung",
    "Nachholen von erforderlichen Vorkenntnissen": "Vorkenntnisnachholung",
    "fehlende soziale Kontakte": "Isolation",
    "Benachteiligung": "Ungleichbehandlung",
    "fehlende Berufsperspektiven": "Karriereunsicherheit"
}


df_ = pd.DataFrame(df, columns=["Category"])

# Apply renaming
df["Category"] = df["Category"].replace(umbenennungen)

In [55]:
fig = px.line_polar(
    df,
    r=df["sum"],
    theta=df["Category"],
    line_close=True
)

# Fill area inside the polygon
fig.update_traces(fill="toself")

# Layout configuration
fig.update_layout(
    template="infoviz",
    title=f"Beeinträchtigende Faktoren im Studium des FB09 in {year}",
    polar=dict(
        radialaxis=dict(
            range=[0, 800]
        )
    )
)

fig.show()
fig.write_image("Plots/pdf/beeintraechtigungen1.pdf")

# Normierte Darstellung:

In [56]:
infoviz_template = dict(
    layout=dict(
        template="plotly_white",
        title=dict(
            font=dict(size=20, family="Arial", weight="bold", color="black"),
            xanchor="left",  
            xref="paper",
            x=0,
            subtitle=dict(
                text=f"normierte, relative Verteilung der Beeinsträchtigungen",
                font=dict(color="gray", size=13),
            ),
        ),
        xaxis=dict(
            showgrid=False,
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelstandoff=10,
            ticklabelposition="outside bottom"
        ),
        yaxis=dict(
            showgrid=True, gridcolor="lightgrey",
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelstandoff=10,
            ticklabelposition="outside left"
        ),
    )
)
pio.templates["infoviz"] = infoviz_template


In [57]:
df = pd.read_excel("data.xlsx", sheet_name=year)

# Filter relevant rows
df = df[
    (df["Variable"] == "Studienbeeinträchtigungen")
    | (df["Category"] == "Anzahl Teilnahmen an Studierendenbefragung")
    | (df["Category"] == "Anteil Teilnehmer (%)")
]

# Remove columns with '*' in the name
df = df[df.columns[~df.columns.str.contains(r"\*")]]
df.columns = df.columns.str.replace("*", "", regex=False)

# Fix typo in column name
df = df.rename(columns={"Master Getränketechnologi": "Master Getränketechnologie"})

# Get number of participants per program
anzahl_studierende = (
    df[df["Category"] == "Anzahl Teilnahmen an Studierendenbefragung"]
    .iloc[:, 2:]
    .values
    .flatten()
)

# Extract and convert relevant part of the data
df_beeintraechtigungen = (
    df[df["Variable"] == "Studienbeeinträchtigungen"]
    .iloc[:, 2:]
    .astype(float)
)

# Convert percentages to absolute numbers
df_relativ = df_beeintraechtigungen.multiply(anzahl_studierende, axis=1) / 100

# Insert category column back
df_relativ.insert(
    0,
    "Category",
    df[df["Variable"] == "Studienbeeinträchtigungen"]["Category"].values
)

# Ensure numeric conversion for any specific column (if needed)
df["Bachelor Agrarwissenschaften"] = pd.to_numeric(
    df["Bachelor Agrarwissenschaften"],
    errors="coerce"
)

# Fill missing values and compute total
df_relativ = df_relativ.fillna(0)
df_relativ["sum"] = df_relativ.sum(axis=1, numeric_only=True)

# Final filtering
df_relativ = df_relativ.loc[
    ~df_relativ["Category"].isin([
        "Anzahl Teilnahmen an Studierendenbefragung",
        "Anteil Teilnehmer (%)"
    ]),
    ["Category", "sum"]
]

/var/folders/9v/2nh1cygd6qv3_k0knr9c4y_80000gn/T/ipykernel_13245/3378800454.py:49: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [58]:
# Clean up category labels by removing ' (%)'
df_relativ["Category"] = df_relativ["Category"].str.replace(" (%)", "", regex=False)

In [59]:
# Define renaming map for clearer category labels
umbenennungen = {
    "zeitliche Überschneidungen": "Zeitmanagement-Probleme",
    "Stundenplanprobleme wegen weiter Wege": "Wegeprobleme",
    "Organisation des Prüfungsablaufes": "Prüfungsorganisation",
    "Engpässe in der Literaturversorgung": "Literaturmangel",
    "kein Labor- / Praktikumsplatz bekommen": "Fehlender Praktikums-/Laborplatz",
    "Anforderungen sind zu hoch": "Überforderung",
    "Leistungsnachweise / Prüfungen nicht geschafft": "Prüfungsprobleme",
    "Job / Erwerbstätigkeit": "Erwerbstätigkeit",
    "familiäre Verpflichtungen": "Familienpflichten",
    "Krankheit / Behinderung": "Gesundheitsprobleme",
    "große Entfernung zwischen Wohnort und JLU": "Entfernung zum Studienort",
    "Doppelstudium": "Doppelte Studienbelastung",
    "Nachholen von erforderlichen Vorkenntnissen": "Vorkenntnisnachholung",
    "fehlende soziale Kontakte": "Isolation",
    "Benachteiligung": "Ungleichbehandlung",
    "fehlende Berufsperspektiven": "Karriereunsicherheit"
}

# Optional: preserve original categories
df_relativ_ = pd.DataFrame(df_relativ, columns=["Category"])

# Apply renaming
df_relativ["Category"] = df_relativ["Category"].replace(umbenennungen)

In [60]:
# Create polar line chart
fig = px.line_polar(
    df_relativ,
    r=df_relativ["sum"],
    theta=df_relativ["Category"],
    line_close=True
)

# Fill the area inside the line
fig.update_traces(fill="toself")

# Layout settings
fig.update_layout(
    template="infoviz",
    title=f"Beeinträchtigende Faktoren im Studium des FB09 in {year}",
    polar=dict(
        radialaxis=dict(showticklabels=False)
    ),
    height=500,
    width=1400
)

fig.show()
fig.write_image("Plots/pdf/beeintraechtigungen2.pdf")